# json파일 전처리

In [3]:
import json
import pandas as pd

json_data = "C:\\Users\\IMS\Desktop\\Hwangsihoon\\WebCam\\participant_data.json"

with open(json_data, 'r') as f:
    data = json.load(f)

# user_name 초기화를 위한 빈 리스트 생성
records = []

# 각 user_name에 대해 meta 정보와 dot_info 정보 추출 및 데이터프레임 생성
for user_name, user_data in data.items():
    meta_info = user_data["meta"]
    dot_info = user_data["dot_info"]
    for n_key, n_value in dot_info.items():
        for m_key, m_value in n_value.items():
            # 각 dot_info 항목에 meta 정보 및 user_name 추가, n, m_key : 파일명 -> User_n_m
            combined_data = {**meta_info, **m_value, "user_name": user_name, "n_key": n_key, "m_key": m_key}
            records.append(combined_data)

df = pd.DataFrame(records)

# 왼쪽, 오른쪽 눈 df화
df_right = df[['user_name', 'file_name_right', 'right_landmarks', 'label']]
df_left = df[['user_name', 'file_name_left', 'left_landmarks', 'label']]


In [4]:
import os
import glob
from keras.preprocessing.image import load_img, img_to_array

import numpy as np

folder_path = "C:\\Users\\IMS\\Desktop\\Hwangsihoon\\WebCam\img"

files = glob.glob(os.path.join(folder_path, '*.jpg'))

# right파일, left파일 각각 분리
right_images = [file for file in files if 'right' in os.path.basename(file).lower()]
left_images = [file for file in files if 'left' in os.path.basename(file).lower()]


def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

right_images = [load_and_preprocess_image(img_path) for img_path in right_images]
left_images = [load_and_preprocess_image(img_path) for img_path in left_images]

label = np.array(df_right['label'].tolist())

print(label.shape)
print(type(label))
print(label)

(31995, 2)
<class 'numpy.ndarray'>
[[0.22424955 0.28144491]
 [0.22424955 0.28144491]
 [0.22424955 0.28144491]
 ...
 [0.33093688 0.87473578]
 [0.33093688 0.87473578]
 [0.33093688 0.87473578]]


# CNN Model

In [15]:
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.metrics import accuracy_score
import tensorflow as tf

x = np.array(right_images)  # 이미지

y = np.array(label)          # 레이블

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())


model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation=None))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 57600)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                3686464   
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [7]:
# import tensorflow as tf

# class R2Score(tf.keras.metrics.Metric):
#     def __init__(self, name='r2_score', **kwargs):
#         super(R2Score, self).__init__(name=name, **kwargs)
#         self.ssr = self.add_weight(name='ssr', initializer='zeros')
#         self.sst = self.add_weight(name='sst', initializer='zeros')
        
#     def update_state(self, y_true, y_pred, sample_weight=None):
#         ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
#         ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
        
#         self.ssr.assign_add(ss_res)
#         self.sst.assign_add(ss_tot)
    
#     def result(self):
#         return 1 - (self.ssr / (self.sst + tf.keras.backend.epsilon()))
    
#     def reset_states(self):
#         self.ssr.assign(0.0)
#         self.sst.assign(0.0)


In [26]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import tensorflow as tf

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])

# 모델 학습
history = model.fit(x_train, y_train, epochs=40, batch_size=256, validation_split=0.4, verbose=2)

# 모델 평가
loss, mae = model.evaluate(x_test, y_test, verbose=0)
print(f'Test Loss: {loss}')
print(f'Test MAE: {mae}')

# 예측
predictions = model.predict(x_test)

# 평가 지표 계산
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

# 결과 출력
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R² Score: {r2}')

# 오차 계산
errors = y_test - predictions

# 예측 값, 실제 값 및 오차 출력 (예시로 첫 5개 데이터만 출력)
for i in range(5):
    print(f'Actual Value:\n{y_test[i]}')
    print(f'Predicted Value:\n{predictions[i]}')
    print(f'Error:\n{errors[i]}\n')

Epoch 1/500
640/640 - 222s - loss: 0.0761 - mean_absolute_error: 0.2347 - val_loss: 0.0942 - val_mean_absolute_error: 0.2609
Epoch 2/500
640/640 - 220s - loss: 0.0740 - mean_absolute_error: 0.2305 - val_loss: 0.1006 - val_mean_absolute_error: 0.2675
Epoch 3/500
640/640 - 219s - loss: 0.0722 - mean_absolute_error: 0.2267 - val_loss: 0.1022 - val_mean_absolute_error: 0.2692
Epoch 4/500
640/640 - 218s - loss: 0.0704 - mean_absolute_error: 0.2231 - val_loss: 0.1061 - val_mean_absolute_error: 0.2723
Epoch 5/500
640/640 - 218s - loss: 0.0682 - mean_absolute_error: 0.2185 - val_loss: 0.1067 - val_mean_absolute_error: 0.2731
Epoch 6/500
640/640 - 219s - loss: 0.0662 - mean_absolute_error: 0.2141 - val_loss: 0.1147 - val_mean_absolute_error: 0.2803
Epoch 7/500
640/640 - 218s - loss: 0.0645 - mean_absolute_error: 0.2105 - val_loss: 0.1175 - val_mean_absolute_error: 0.2829
Epoch 8/500
640/640 - 218s - loss: 0.0622 - mean_absolute_error: 0.2057 - val_loss: 0.1232 - val_mean_absolute_error: 0.2879


640/640 - 218s - loss: 0.0310 - mean_absolute_error: 0.1260 - val_loss: 0.1633 - val_mean_absolute_error: 0.3276
Epoch 67/500
640/640 - 218s - loss: 0.0308 - mean_absolute_error: 0.1253 - val_loss: 0.1646 - val_mean_absolute_error: 0.3290
Epoch 68/500
640/640 - 218s - loss: 0.0306 - mean_absolute_error: 0.1244 - val_loss: 0.1757 - val_mean_absolute_error: 0.3387
Epoch 69/500
640/640 - 217s - loss: 0.0306 - mean_absolute_error: 0.1247 - val_loss: 0.1723 - val_mean_absolute_error: 0.3361
Epoch 70/500
640/640 - 217s - loss: 0.0305 - mean_absolute_error: 0.1244 - val_loss: 0.1702 - val_mean_absolute_error: 0.3342
Epoch 71/500
640/640 - 217s - loss: 0.0305 - mean_absolute_error: 0.1240 - val_loss: 0.1656 - val_mean_absolute_error: 0.3299
Epoch 72/500
640/640 - 217s - loss: 0.0302 - mean_absolute_error: 0.1235 - val_loss: 0.1704 - val_mean_absolute_error: 0.3335
Epoch 73/500
640/640 - 216s - loss: 0.0299 - mean_absolute_error: 0.1229 - val_loss: 0.1663 - val_mean_absolute_error: 0.3296
Epoch

Epoch 131/500
640/640 - 212s - loss: 0.0262 - mean_absolute_error: 0.1090 - val_loss: 0.1799 - val_mean_absolute_error: 0.3418
Epoch 132/500
640/640 - 212s - loss: 0.0260 - mean_absolute_error: 0.1080 - val_loss: 0.1904 - val_mean_absolute_error: 0.3512
Epoch 133/500
640/640 - 212s - loss: 0.0260 - mean_absolute_error: 0.1079 - val_loss: 0.1889 - val_mean_absolute_error: 0.3496
Epoch 134/500
640/640 - 210s - loss: 0.0262 - mean_absolute_error: 0.1086 - val_loss: 0.1888 - val_mean_absolute_error: 0.3489
Epoch 135/500
640/640 - 211s - loss: 0.0259 - mean_absolute_error: 0.1080 - val_loss: 0.1815 - val_mean_absolute_error: 0.3423
Epoch 136/500
640/640 - 212s - loss: 0.0258 - mean_absolute_error: 0.1075 - val_loss: 0.1912 - val_mean_absolute_error: 0.3506
Epoch 137/500
640/640 - 212s - loss: 0.0259 - mean_absolute_error: 0.1079 - val_loss: 0.1889 - val_mean_absolute_error: 0.3496
Epoch 138/500
640/640 - 215s - loss: 0.0259 - mean_absolute_error: 0.1079 - val_loss: 0.1869 - val_mean_absolut

Epoch 196/500
640/640 - 213s - loss: 0.0245 - mean_absolute_error: 0.1018 - val_loss: 0.1901 - val_mean_absolute_error: 0.3498
Epoch 197/500
640/640 - 213s - loss: 0.0244 - mean_absolute_error: 0.1012 - val_loss: 0.1904 - val_mean_absolute_error: 0.3507
Epoch 198/500
640/640 - 213s - loss: 0.0244 - mean_absolute_error: 0.1012 - val_loss: 0.1916 - val_mean_absolute_error: 0.3507
Epoch 199/500
640/640 - 212s - loss: 0.0243 - mean_absolute_error: 0.1009 - val_loss: 0.1904 - val_mean_absolute_error: 0.3495
Epoch 200/500
640/640 - 212s - loss: 0.0245 - mean_absolute_error: 0.1013 - val_loss: 0.1996 - val_mean_absolute_error: 0.3567
Epoch 201/500
640/640 - 213s - loss: 0.0243 - mean_absolute_error: 0.1003 - val_loss: 0.1895 - val_mean_absolute_error: 0.3490
Epoch 202/500
640/640 - 213s - loss: 0.0242 - mean_absolute_error: 0.1004 - val_loss: 0.1973 - val_mean_absolute_error: 0.3558
Epoch 203/500
640/640 - 214s - loss: 0.0243 - mean_absolute_error: 0.1006 - val_loss: 0.1898 - val_mean_absolut

Epoch 261/500
640/640 - 220s - loss: 0.0236 - mean_absolute_error: 0.0971 - val_loss: 0.1952 - val_mean_absolute_error: 0.3542
Epoch 262/500
640/640 - 219s - loss: 0.0236 - mean_absolute_error: 0.0971 - val_loss: 0.2051 - val_mean_absolute_error: 0.3631
Epoch 263/500
640/640 - 219s - loss: 0.0235 - mean_absolute_error: 0.0971 - val_loss: 0.1925 - val_mean_absolute_error: 0.3523
Epoch 264/500
640/640 - 219s - loss: 0.0235 - mean_absolute_error: 0.0972 - val_loss: 0.1887 - val_mean_absolute_error: 0.3495
Epoch 265/500
640/640 - 220s - loss: 0.0236 - mean_absolute_error: 0.0975 - val_loss: 0.1907 - val_mean_absolute_error: 0.3507
Epoch 266/500
640/640 - 219s - loss: 0.0234 - mean_absolute_error: 0.0966 - val_loss: 0.1995 - val_mean_absolute_error: 0.3587
Epoch 267/500
640/640 - 219s - loss: 0.0235 - mean_absolute_error: 0.0968 - val_loss: 0.1894 - val_mean_absolute_error: 0.3498
Epoch 268/500
640/640 - 219s - loss: 0.0236 - mean_absolute_error: 0.0968 - val_loss: 0.1954 - val_mean_absolut

Epoch 326/500
640/640 - 216s - loss: 0.0231 - mean_absolute_error: 0.0947 - val_loss: 0.1912 - val_mean_absolute_error: 0.3508
Epoch 327/500
640/640 - 217s - loss: 0.0231 - mean_absolute_error: 0.0946 - val_loss: 0.1920 - val_mean_absolute_error: 0.3513
Epoch 328/500
640/640 - 218s - loss: 0.0230 - mean_absolute_error: 0.0938 - val_loss: 0.1962 - val_mean_absolute_error: 0.3556
Epoch 329/500
640/640 - 219s - loss: 0.0231 - mean_absolute_error: 0.0947 - val_loss: 0.1997 - val_mean_absolute_error: 0.3582
Epoch 330/500
640/640 - 218s - loss: 0.0231 - mean_absolute_error: 0.0942 - val_loss: 0.2022 - val_mean_absolute_error: 0.3604
Epoch 331/500
640/640 - 217s - loss: 0.0231 - mean_absolute_error: 0.0952 - val_loss: 0.1968 - val_mean_absolute_error: 0.3555
Epoch 332/500
640/640 - 217s - loss: 0.0231 - mean_absolute_error: 0.0947 - val_loss: 0.1832 - val_mean_absolute_error: 0.3434
Epoch 333/500
640/640 - 217s - loss: 0.0231 - mean_absolute_error: 0.0944 - val_loss: 0.1947 - val_mean_absolut

Epoch 391/500
640/640 - 217s - loss: 0.0228 - mean_absolute_error: 0.0927 - val_loss: 0.1841 - val_mean_absolute_error: 0.3452
Epoch 392/500
640/640 - 218s - loss: 0.0227 - mean_absolute_error: 0.0929 - val_loss: 0.2035 - val_mean_absolute_error: 0.3613
Epoch 393/500
640/640 - 217s - loss: 0.0227 - mean_absolute_error: 0.0926 - val_loss: 0.1942 - val_mean_absolute_error: 0.3531
Epoch 394/500
640/640 - 217s - loss: 0.0227 - mean_absolute_error: 0.0928 - val_loss: 0.1986 - val_mean_absolute_error: 0.3566
Epoch 395/500
640/640 - 217s - loss: 0.0228 - mean_absolute_error: 0.0932 - val_loss: 0.2017 - val_mean_absolute_error: 0.3593
Epoch 396/500
640/640 - 216s - loss: 0.0226 - mean_absolute_error: 0.0921 - val_loss: 0.1924 - val_mean_absolute_error: 0.3521
Epoch 397/500
640/640 - 217s - loss: 0.0227 - mean_absolute_error: 0.0927 - val_loss: 0.1970 - val_mean_absolute_error: 0.3562
Epoch 398/500
640/640 - 217s - loss: 0.0227 - mean_absolute_error: 0.0925 - val_loss: 0.2028 - val_mean_absolut

Epoch 456/500
640/640 - 219s - loss: 0.0226 - mean_absolute_error: 0.0915 - val_loss: 0.1942 - val_mean_absolute_error: 0.3539
Epoch 457/500
640/640 - 219s - loss: 0.0226 - mean_absolute_error: 0.0916 - val_loss: 0.1956 - val_mean_absolute_error: 0.3550
Epoch 458/500
640/640 - 219s - loss: 0.0225 - mean_absolute_error: 0.0913 - val_loss: 0.2002 - val_mean_absolute_error: 0.3588
Epoch 459/500
640/640 - 219s - loss: 0.0225 - mean_absolute_error: 0.0910 - val_loss: 0.1936 - val_mean_absolute_error: 0.3535
Epoch 460/500
640/640 - 219s - loss: 0.0225 - mean_absolute_error: 0.0913 - val_loss: 0.1923 - val_mean_absolute_error: 0.3525
Epoch 461/500
640/640 - 219s - loss: 0.0225 - mean_absolute_error: 0.0917 - val_loss: 0.2012 - val_mean_absolute_error: 0.3598
Epoch 462/500
640/640 - 218s - loss: 0.0224 - mean_absolute_error: 0.0912 - val_loss: 0.1971 - val_mean_absolute_error: 0.3561
Epoch 463/500
640/640 - 219s - loss: 0.0224 - mean_absolute_error: 0.0908 - val_loss: 0.2002 - val_mean_absolut